In [1]:
####>> Step 1 <<####
from pyspark.sql import SparkSession
from pyspark import SparkContext
import pyspark
from pyspark.sql.functions import *
## ------------------------------ ##

In [2]:
####>> Step 2 <<####
###### SparkSession ######
def session_spark():
    spark = (
        SparkSession.builder
            .master("local[*]")
            .appName("minio")
            ## Config Fields
            .config('spark.sql.debug.maxToStringFields', 500)
            .config('spark.debug.maxToStringFields', 500)
            # Optimize
            .config("delta.autoOptimize.optimizeWrite", "true")
            .config("delta.autoOptimize.autoCompact", "true")
            ## Delta Table
            .config("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0")
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
            ## MinIO
            .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
            .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
            .config("spark.hadoop.fs.s3a.secret.key", "minioadmin")
            .config("spark.hadoop.fs.s3a.path.style.access", "true")
            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
            ## Jars
            .config("spark.jars", "C:/Spark-3.3.2/jars/aws-java-sdk-bundle-1.11.1026.jar, \
                                   C:/Spark-3.3.2/jars/hadoop-aws-3.3.2.jar")
            ## Hive SQL
            .enableHiveSupport()
            .getOrCreate()
    )
    return spark

In [3]:
####>> Step 3 <<####
spark = session_spark()
print("conexão com o Spark - ok!!!\n")
print("Version:", spark.version)
## ------------------------------ ##

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [9]:
####>> Step 4 <<####
log4jLogger = spark.sparkContext._jvm.org.apache.log4j
LOGGER = log4jLogger.LogManager.getLogger(__name__)
spark.sparkContext.setLogLevel("ERROR")
## ------------------------------ ##

NameError: name 'spark' is not defined

In [ ]:
####>> Step 5 <<####
spark.conf.set("spark.sql.shuffle.partitions", spark.sparkContext.defaultParallelism)
## ------------------------------ ##

In [ ]:
####>> Step 6 <<####
from delta import *
## ------------------------------ ##

In [ ]:
####>> Step 7 <<####
targetBucket = "bronze"
sourcePath = f"C:/Lenzi/Spark/LearningSparkV2-master/chapter3/data/sf-fire-calls.csv"
targetPath = f"s3a://{targetBucket}/"
## ------------------------------ ##

In [ ]:
####>> Step 8 <<####
# dfRaw = read_json(spark, sourcePath, "true", "true", schema_evolution(sourcePath, "json"))
fire_df = spark.read.csv(sourcePath, header=True, inferSchema=True)

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

data2 = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=data2,schema=schema)

In [ ]:
####>> Step 9 <<####
print(df.count())
## ------------------------------ ##

In [ ]:
####>> Step 10 <<####
(
   df.write      
      .format("delta")
      .mode("overwrite")
      .option("overwriteSchema", "true")
      .option("mergeSchema", "true")
      .option("path", targetPath)
      .save()
)
#------------------------------#

In [ ]:
spark.stop()